In [4]:
conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia -y
conda install -c conda-forge tensorboard

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt install -y git-lfs -y
!git lfs install

In [5]:
!nvidia-smi


Wed Jan  4 12:20:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import torch
#torch.cuda.is_available()
#torch.cuda.device_count()
#torch.cuda.current_device()
torch.cuda.get_device_name(0)


'Tesla T4'

In [7]:
# !add-apt-repository -y ppa:jonathonf/ffmpeg-4
# !apt update
# !apt install -y ffmpeg


/bin/bash: add-apt-repository: command not found
Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 http://deb.debian.org/debian buster-updates InRelease
Hit:3 http://security.debian.org/debian-security buster/updates InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
57 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.1.10-0+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.


In [8]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install evaluate>=0.30
# !pip install jiwer
# !pip install gradio


Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-0gwlc4_g
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0gwlc4_g
  Resolved https://github.com/huggingface/transformers to commit d7b66d9b44a13489d3c86d6732d9a98b3059c0e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'
Keyring is skipped due to an exception: 'keyring.backends'


In [9]:
from huggingface_hub import notebook_login

notebook_login()


In [10]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", use_auth_token=True)

print(common_voice)


Found cached dataset common_voice_11_0 (/root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/hi/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)
Found cached dataset common_voice_11_0 (/root/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/hi/11.0.0/f8e47235d9b4e68fa24ed71d63266a02018ccf7194b2a8c9c598a5f3ab304d9f)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 6540
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 2894
    })
})


In [11]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])


In [12]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")


In [13]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")


In [14]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Input:                 हमने उसका जन्मदिन मनाया।
Decoded w/ special:    <|startoftranscript|><|hi|><|transcribe|><|notimestamps|>हमने उसका जन्मदिन मनाया।<|endoftext|>
Decoded w/out special: हमने उसका जन्मदिन मनाया।
Are equal:             True


In [15]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")


In [16]:
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/fd162a4428db74a3e1004d3f4a1eb432236f436ba9cf5e0b931e9fc47942210c/common_voice_hi_26008353.mp3', 'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0710508e-07,
       4.4680878e-08, 2.2291848e-09], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


In [17]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


In [18]:
print(common_voice["train"][0])


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/fd162a4428db74a3e1004d3f4a1eb432236f436ba9cf5e0b931e9fc47942210c/common_voice_hi_26008353.mp3', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -7.5403413e-08,  2.0818506e-07,  9.6670441e-08], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'हमने उसका जन्मदिन मनाया।'}


In [19]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [20]:
# common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=4)


In [21]:
# common_voice.save_to_disk("hi-common-voice")


In [43]:
from datasets import load_from_disk
common_voice = load_from_disk("hi-common-voice")


/opt/conda/lib/python3.7/site-packages/datasets/dataset_dict.py:1244: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  FutureWarning,


In [13]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

bucket = sess.default_bucket() 
bucket

'sagemaker-us-east-1-348052051973'

In [15]:
%%bash

aws s3 cp --recursive hi-common-voice s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice


upload: hi-common-voice/dataset_dict.json to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/dataset_dict.json
upload: hi-common-voice/test/data-00001-of-00006.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/data-00001-of-00006.arrow
upload: hi-common-voice/test/data-00000-of-00006.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/data-00000-of-00006.arrow
upload: hi-common-voice/test/dataset_info.json to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/dataset_info.json
upload: hi-common-voice/test/state.json to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/state.json
upload: hi-common-voice/test/data-00003-of-00006.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/data-00003-of-00006.arrow
upload: hi-common-voice/test/data-00002-of-00006.arrow to s3://sagemaker-us-east-1-348052051973/whisper/data/hi-common-voice/test/data-0

In [23]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [24]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [25]:
import evaluate

metric = evaluate.load("wer")


In [26]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [27]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [28]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [29]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
    use_cache=False,
)


In [44]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


/root/./whisper-small-hi is already a clone of https://huggingface.co/bubuxiong/whisper-small-hi. Make sure you pull the latest changes with `repo.git_pull()`.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [ ]:
trainer.train()


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6540
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.087900,0.290848,33.793279
2000,0.021600,0.344039,33.022941


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

In [ ]:
trainer.save_model('whisper-small-hi.model')

In [10]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperTokenizer

model = WhisperForConditionalGeneration.from_pretrained("bubuxiong/whisper-small-hi")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")


In [12]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(task="automatic-speech-recognition", model=model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor)  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch(share=True)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://5e8f1114-08ff-4b0d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


/opt/conda/lib/python3.7/site-packages/gradio/processing_utils.py:225: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))
/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1239: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [59]:
pip install transformers[sentencepiece]


Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.120.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 5.1.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
